In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import when
from pyspark.sql.functions import col
from pyspark.sql.types import * 
from pyspark.sql.functions import lit
import pyodbc
import pandas as pd
import pyspark.sql.functions as sf
from datetime import date, timedelta
import datetime

In [2]:
spark = SparkSession.builder.config("spark.driver.memory", "15g").getOrCreate()

In [3]:
def generate_time_range_data(startdate,enddate):
    format = '%Y-%m-%d'
    day_time = []
    startdate = datetime.datetime.strptime(startdate, format).date()
    enddate = datetime.datetime.strptime(enddate, format).date()
    delta = enddate - startdate
    for i in range(delta.days + 1):
        day = startdate + timedelta(days=i)
        day = day.strftime('%Y%m%d')
        day_time.append(str(day))
    return day_time

In [4]:
day_range = generate_time_range_data('2022-06-01','2022-06-14')
path = "C:\\Users\\ASUS\\OneDrive\\Big_Data_Analytics\\BigData_Gen1\\Class 4\\log_search\\"
df = spark.read.parquet('C:\\Users\\ASUS\\OneDrive\\Big_Data_Analytics\\BigData_Gen1\\Class 4\\log_search\\20220601')
df = df.filter(df.user_id.isNotNull())
df = df.select('datetime','user_id','keyword')
result_t6 = df
for i in day_range[1:]:
    df1 = spark.read.parquet(path+i)
    df1 = df1.filter(df1.user_id.isNotNull())
    df1 = df1.select('datetime','user_id','keyword')
    result_t6 = result_t6.union(df1)
    
    

In [5]:
result_t6 = result_t6.drop(result_t6.datetime)
result_t6 = result_t6.filter(result_t6.keyword.isNotNull())



In [6]:
result_t6.groupBy('keyword').count().orderBy(col('count').desc()).show(truncate=False)

+--------------------------------------------+-----+
|keyword                                     |count|
+--------------------------------------------+-----+
|Liên Minh Công Lý: Phiên bản của Zack Snyder|8741 |
|fairy tail                                  |6771 |
|thiên nga bóng đêm                          |5471 |
|sao băng                                    |4909 |
|nữ thanh tra tài ba                         |4895 |
|bắt ma phá án                               |4837 |
|siêu nhân                                   |3938 |
|mộng hoa lục                                |3411 |
|cuộc chiến thượng lưu                       |3001 |
|cô nàng trong trắng oh woo ri               |2950 |
|vô tình nhặt được tổng tài                  |2646 |
|hội pháp sư                                 |2556 |
|tôi thấy hoa vàng trên cỏ xanh              |2540 |
|why her?                                    |2449 |
|lấy danh nghĩa người nhà                    |2443 |
|bạn trai tôi là hồ ly                       |

In [16]:
day_range = generate_time_range_data('2022-07-01','2022-07-14')
path = "C:\\Users\\ASUS\\OneDrive\\Big_Data_Analytics\\BigData_Gen1\\Class 4\\log_search\\"
df = spark.read.parquet('C:\\Users\\ASUS\\OneDrive\\Big_Data_Analytics\\BigData_Gen1\\Class 4\\log_search\\20220701')
df = df.filter(df.user_id.isNotNull())
df = df.select('datetime','user_id','keyword')
result_t7 = df
for i in day_range[1:]:
    df1 = spark.read.parquet(path+i)
    df1 = df1.filter(df1.user_id.isNotNull())
    df1 = df1.select('datetime','user_id','keyword')
    result_t7 = result_t7.union(df1)

    
    


In [17]:
result_t7 = result_t7.drop(result_t7.datetime)
result_t7 = result_t7.filter(result_t7.keyword.isNotNull())


In [18]:
result_t7.groupBy('keyword').count().orderBy(col('count').desc()).show(truncate=False)


+--------------------------------------------+-----+
|keyword                                     |count|
+--------------------------------------------+-----+
|Liên Minh Công Lý: Phiên bản của Zack Snyder|8774 |
|fairy tail                                  |5596 |
|thanh thanh tử khâm                         |4465 |
|con tim sắt đá                              |4412 |
|chàng hậu                                   |4096 |
|thiên nga bóng đêm                          |3712 |
|siêu nhân                                   |3638 |
|yêu tinh                                    |3423 |
|vẻ đẹp đích thực                            |3394 |
|hội pháp sư                                 |2911 |
|Tìm kiếm bằng giọng nói                     |2837 |
|kẻ trộm mặt trăng: minions                  |2822 |
|nhất kiến khuynh tâm                        |2802 |
|tôi thấy hoa vàng trên cỏ xanh              |2743 |
|Thử Thách Thần Tượng                        |2593 |
|sao băng                                    |

In [9]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank
from pyspark.sql.functions import col

In [10]:
window = Window.partitionBy("user_id").orderBy(col('keyword').desc())

In [19]:
rank_result_t6 = result_t6.withColumn('RANK',rank().over(window))

In [20]:
rank_result_t7 = result_t7.withColumn('RANK',rank().over(window))

In [21]:
rank_result_t6.show()

+-------+--------------------+----+
|user_id|             keyword|RANK|
+-------+--------------------+----+
|0000402|xem phim nhan duy...|   1|
|0000402|tieu nuong tu nha...|   2|
|0027835|lời nói dối của h...|   1|
|0027835|  lời nói dối của hồ|   2|
|0027835|   lời nói dối của h|   3|
|0027835|    lời nói dối của |   4|
|0027835|     hạ cánh nơi anh|   5|
|0027835|             another|   6|
|0042335| tình anh quang dũng|   1|
|0042335|            tình anh|   2|
|0042335|pháp y tần minh: ...|   3|
|0047402|                  aq|   1|
|0062340|    quân tử lập thân|   1|
|0062340|     lạp tội đồ giám|   2|
|0062340|     lạp tội đồ giám|   2|
|0145732|       again my life|   1|
|0156279|           PLUNDERER|   1|
|0156279|           PLUNDERER|   1|
|0156279|           PLUNDERER|   1|
|0156279|           PLUNDERER|   1|
+-------+--------------------+----+
only showing top 20 rows



In [25]:
rank_result_t6.filter(rank_result_t6.user_id == '0027835').show(truncate=False)

+-------+---------------------+----+
|user_id|keyword              |RANK|
+-------+---------------------+----+
|0027835|lời nói dối của hồ ly|1   |
|0027835|lời nói dối của hồ   |2   |
|0027835|lời nói dối của h    |3   |
|0027835|lời nói dối của      |4   |
|0027835|hạ cánh nơi anh      |5   |
|0027835|another              |6   |
+-------+---------------------+----+



In [28]:
rank_result_t6 = rank_result_t6.filter(rank_result_t6.RANK == '1')

In [29]:
rank_result_t7 = rank_result_t7.filter(rank_result_t7.RANK == '1')

In [35]:
rank_result_t6 = rank_result_t6.distinct()

In [37]:
rank_result_t6.show(truncate=False)

+-------+--------------------------------------------+----+
|user_id|keyword                                     |RANK|
+-------+--------------------------------------------+----+
|0000402|xem phim nhan duyen tien dinh thailan       |1   |
|0027835|lời nói dối của hồ ly                       |1   |
|0042335|tình anh quang dũng                         |1   |
|0047402|aq                                          |1   |
|0062340|quân tử lập thân                            |1   |
|0145732|again my life                               |1   |
|0156279|PLUNDERER                                   |1   |
|0164517|lee min ho                                  |1   |
|016773 |Liên Minh Công Lý: Phiên bản của Zack Snyder|1   |
|0201199|vagabond                                    |1   |
|021135 |hồ sơ trinh sát                             |1   |
|0221207|thư ký kim sa                              |1   |
|0258539|animal                                      |1   |
|0286702|đại mạc đao                    

In [36]:
rank_result_t7 = rank_result_t7.distinct()

In [38]:
rank_result_t7.show(truncate=False)

+--------+-----------------------------------------------------+----+
|user_id |keyword                                              |RANK|
+--------+-----------------------------------------------------+----+
|0000699 |thả thí                                              |1   |
|0013936 |one pice                                             |1   |
|0027835 |minh lan truyện                                      |1   |
|0077738 |penthouse 3                                          |1   |
|0152453 |d day                                                |1   |
|0162317 |dhoom3                                               |1   |
|0197583 |thap nhi dam                                         |1   |
|0201199 |người thầy y đức 2                                   |1   |
|0209346 |công tố viên chuyển sinh                             |1   |
|0258467 |vẻ đẹp đích thực vietsub                             |1   |
|0276520 |bất động sản Trường Sa                               |1   |
|0292234 |DOREMO    

In [40]:
result_t7 = rank_result_t7.toPandas()

In [47]:
import mysql.connector
import pandas as pd
from sqlalchemy import create_engine

In [48]:
user = 'root'
password =''
host = 'localhost'
database = 'Data_Engineer'
port = 3306
url = 'mysql+mysqlconnector://{}:{}@{}:{}/{}'.format(user, password, host, port, database)
engine = create_engine(url, echo=False)
result_t7.to_sql(name='search_behaviour', con=engine, if_exists='append', chunksize=1000, index=False, method='multi')

In [49]:
#cnx = mysql.connector.connect(user=user,password=password,host=host,database=database)
#cursor = cnx.cursor()
#sql = """Insert into customer360 (Contract,Date,SportDuration,TVDuration,ChildDuration,RelaxDuration,MovieDuration,Most_Watch) Values (%s,%s,%s,%s,%s,%s,%s,%s)"""
#cursor.executemany(sql,test.values.tolist())
#cnx.commit()
#print("Import Successfully")

In [53]:
cnx = mysql.connector.connect(user=user,password=password,host=host,database=database)

In [55]:
customer360 = pd.read_sql("select * from customer360", cnx)

In [56]:
customer360

,Contract,Date,SportDuration,TVDuration,ChildDuration,RelaxDuration,MovieDuration,Most_Watch
0,AGAAA0520,2022-04-01,NaN,173458.0,NaN,NaN,447.0,Truyền Hình
1,AGAAA0663,2022-04-01,NaN,89001.0,NaN,NaN,NaN,Truyền Hình
2,AGAAA1058,2022-04-01,NaN,118784.0,NaN,NaN,NaN,Truyền Hình
3,AGAAA1238,2022-04-01,NaN,163158.0,NaN,NaN,NaN,Truyền Hình
4,AGAAA1572,2022-04-01,NaN,27309.0,NaN,NaN,NaN,Truyền Hình
...,...,...,...,...,...,...,...,...
1995,DNH055557,2022-04-01,NaN,1342851.0,NaN,NaN,NaN,Truyền Hình
1996,DNH055743,2022-04-01,NaN,20794.0,1533.0,NaN,NaN,Truyền Hình
1997,DNH055859,2022-04-01,NaN,73500.0,NaN,NaN,NaN,Truyền Hình
1998,DNH055977,2022-04-01,NaN,11006.0,NaN,NaN,NaN,Truyền Hình


In [60]:
log_search_summary = result_t7[0:2000]

In [63]:
final_data = pd.concat([customer360,log_search_summary],axis = 1 )

In [66]:
final_data.drop(columns=['user_id','RANK'],inplace=True)

In [69]:
final_data.rename(columns={'keyword':'Most_Searched'},inplace=True)

In [73]:
final_data

,Contract,Date,SportDuration,TVDuration,ChildDuration,RelaxDuration,MovieDuration,Most_Watch,Most_Searched
0,AGAAA0520,2022-04-01,NaN,173458.0,NaN,NaN,447.0,Truyền Hình,thả thí
1,AGAAA0663,2022-04-01,NaN,89001.0,NaN,NaN,NaN,Truyền Hình,one pice
2,AGAAA1058,2022-04-01,NaN,118784.0,NaN,NaN,NaN,Truyền Hình,minh lan truyện
3,AGAAA1238,2022-04-01,NaN,163158.0,NaN,NaN,NaN,Truyền Hình,penthouse 3
4,AGAAA1572,2022-04-01,NaN,27309.0,NaN,NaN,NaN,Truyền Hình,d day
...,...,...,...,...,...,...,...,...,...
1995,DNH055557,2022-04-01,NaN,1342851.0,NaN,NaN,NaN,Truyền Hình,phim kiếm hiệp
1996,DNH055743,2022-04-01,NaN,20794.0,1533.0,NaN,NaN,Truyền Hình,đấu pha thung khung
1997,DNH055859,2022-04-01,NaN,73500.0,NaN,NaN,NaN,Truyền Hình,vietnam malaisia
1998,DNH055977,2022-04-01,NaN,11006.0,NaN,NaN,NaN,Truyền Hình,eve


In [72]:
#final_data.to_csv('C:\\Users\\ASUS\\OneDrive\\Big_Data_Analytics\\BigData_Gen1\\Class 7\\final_output.csv')